# Init Bionic VTOL

In [ ]:
include("../Flyonic.jl");
using .Flyonic;

using Rotations; # used for initial position

using ReinforcementLearning;
using StableRNGs;
using Flux;
using Flux.Losses;
using Random;
using IntervalSets;
using LinearAlgebra;
using Distributions;

using Plots;
using Statistics;

using BSON: @save, @load # save mode

In [ ]:
create_visualization();

In [ ]:
# indicates how many threads Julia was started with. This is important for the multi-threaded environment
Threads.nthreads()

In [ ]:
eth_vtol_param["gravity"] = 9.81;

In [ ]:
# TODO: All this stuff must be replaced later by your guiding paths.

DESIRED_x = [-4.0, 0.0, 4.0] # desired distance    
angle = calculateAngle([0.0 ,0.0, 1.0], DESIRED_x) # 
DESIRED_R = Matrix(UnitQuaternion(RotY(angle)*RotX(pi/2.0)*RotZ(pi/2.0)))

#create_VTOL("fixgoal", actuators = false, color_vec=[0.0; 1.0; 0.0; 1.0]);
#set_transform("fixgoal", DESIRED_x ,QuatRotation(DESIRED_R)); 

# Create Reinforcement Learning Environment

In [ ]:
mutable struct VtolEnv{A,T,ACT,R<:AbstractRNG} <: AbstractEnv # Parametric Constructor for a subtype of AbstractEnv
    action_space::A
    observation_space::Space{Vector{ClosedInterval{T}}}
    state::Vector{T}
    action::ACT
    done::Bool
    t::T
    rng::R

    name::String #for multible environoments
    visualization::Bool
    realtime::Bool
    
    # Everything you need aditionaly can also go in here.
    x_W::Vector{T}
    v_B::Vector{T}
    R_W::Matrix{T}
    ω_B::Vector{T}
    wind_W::Vector{T}
    Δt::T
    
    # Current Bonus / Target
    x_d_W::Vector{T}
    R_d_W::Matrix{T}

    desired_length::T
    desired_angle::T

    # NEW
    covered_line::T
    previously_covered_line::T
    reached_goal::Bool

    # MULTIPOINT
    n_points::Int
    x_d_W_all::Matrix{T}
    R_d_W_all::Matrix{T}

    current_point::Int
    cummulative_covered_line::T
end

In [ ]:
# define a keyword-based constructor for the type declared in the mutable struct typedef. 
# It could also be done with the macro Base.@kwdef.
function VtolEnv(;
     
    #continuous = true,
    rng = Random.GLOBAL_RNG, # Random number generation
    name = "vtol",
    visualization = false,
    realtime = false,
    kwargs... # let the function take an arbitrary number of keyword arguments 
)
    
    T = Float64; # explicit type which is used e.g. in state. Cannot be altered due to the poor matrix defininon.

    #action_space = Base.OneTo(21) # 21 discrete positions for the flaps
    
    action_space = Space(
        ClosedInterval{T}[
            0.0..2.0, # propeller 1
            0.0..2.0, # propeller 2
            ], 
    )

    
    state_space = Space( # Three continuous values in state space.
        ClosedInterval{T}[
            
            # If you are not flying horizontally, you can later switch gravitation 
            # back on and counteract it with the rotors as well.
            # In addition, once the drone has flown over its target, 
            # it can "fall down" and does not have to turn around.
            
            # orientate yourself on the state space from the paper
            typemin(T)..typemax(T), # position along x
            typemin(T)..typemax(T), # position along z
            typemin(T)..typemax(T), # orientation along x
            typemin(T)..typemax(T), # orientation along z
            typemin(T)..typemax(T), # velocity along x BODY coordinates
            typemin(T)..typemax(T), # velocity along y BODY coordinates
            typemin(T)..typemax(T), # rotational velocity along z BODY coordinates
            
            typemin(T)..typemax(T), # position error along x
            typemin(T)..typemax(T), # position error along z
            # Not used in Paper!!!
            typemin(T)..typemax(T), # target rotation along x (better than angle for neural networks)
            typemin(T)..typemax(T), # target rotation along z (better than angle for neural networks)
            
            # NEW 
            typemin(T)..typemax(T), # The distance along the connecting line which has been passed
            typemin(T)..typemax(T), # The distance along the connecting line which has been previously passed         
            ], 
    )
    
    if visualization
        create_VTOL(name, actuators = true, color_vec=[1.0; 1.0; 0.6; 1.0]);
    end

    environment = VtolEnv(
        action_space,
        state_space,
        zeros(T, 13), # current state, needs to be extended. 
        rand(action_space),
        false, # episode done ?
        0.0, # time
        rng, # random number generator  
        name,
        visualization,
        realtime,
        zeros(T, 3), # x_W
        zeros(T, 3), # v_B
        #Matrix(UnitQuaternion((RotX(pi)))),
        [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], # Float64... so T needs to be Float64
        zeros(T, 3), # ω_B
        zeros(T, 3), # wind_W
        T(0.025), # Δt 
        # TODO Random
        DESIRED_x, # desired distance 
        [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0], # desired orientation
        
        0.0,    # Desired distance
        0.0,    # Desired angle
    
        0.0, # Covered line
        0.0,    # Previously covered line
        false,  # Reached Goal

        2,
        zeros(3, 2),
        zeros(3, 2*3),
        0,
        0.0
    )
    
    
    reset!(environment)
    
    return environment
    
end;

Just for explanation:

1. A mutable Struct is created. A struct is a constructor and a constructor is a function that creates new objects.
2. A outer keyword-based constructor method is added for the type declared in the mutable struct typedef before.

So now we have a function with two methods. Julia will decide which method to call by multiple dispatch.

In [ ]:
methods(VtolEnv)

# Define the RL interface

In [ ]:
Random.seed!(env::VtolEnv, seed) = Random.seed!(env.rng, seed)
RLBase.action_space(env::VtolEnv) = env.action_space
RLBase.state_space(env::VtolEnv) = env.observation_space
RLBase.is_terminated(env::VtolEnv) = env.done
RLBase.state(env::VtolEnv) = env.state

In [ ]:
function computeReward(env::VtolEnv{A,T}) where {A,T}
    
    # TODO: Add tolerance for VTOL-Drone
    if norm(env.x_W - env.x_d_W) < 1 && env.reached_goal == false
        near_goal = 100#exp(-norm(env.x_W - env.x_d_W))*10
        env.reached_goal = true
    else
        near_goal = 0
    end

    distance_goal = 0#norm(env.x_W - env.x_d_W) * 20
    
    limit_rotation = 0.1 * env.ω_B[3]^2 #* 10.0

    if env.covered_line > 1
        new_progress = -(env.covered_line-env.previously_covered_line)*100
        progress = - env.covered_line + env.cummulative_covered_line#* 10#sign(env.covered_line)*abs(env.covered_line-norm(env.x_d_W))*10
    else
        new_progress = (env.covered_line-env.previously_covered_line)*100
        progress = env.covered_line + env.cummulative_covered_line #* 10
    end
    # TODO: Make yourself comfortable with what this is
    difference_angle = sum((env.R_W[:,1] - env.R_d_W[:,1]).^2)
    
    
    #difference_angle = abs(env.state[3])*50.0

    #distance_goal = norm(env.x_d_W-[env.state[1], env.state[2], 0])*100.0

    #difference_angle = abs(env.state[3]-env.angle_d_W)*50.0
    

    # TODO Save last position or last projection somewhere (env.last) --> Compare 
    # to current project along line
    #print(difference_angle)
    #not_upright_orientation = abs(env.state[1]-pi*0.5)*10.0
    #not_centered_position = abs(env.state[2])*10.0
    #hight = env.state[4]*100.0

    env.previously_covered_line = env.covered_line
    
    return near_goal - distance_goal + progress + new_progress - limit_rotation - difference_angle
end


RLBase.reward(env::VtolEnv{A,T}) where {A,T} = computeReward(env)

In [ ]:
function RLBase.reset!(env::VtolEnv{A,T}) where {A,T}
    
    # Visualize initial state
    if env.visualization
        set_transform(env.name, env.x_W, QuatRotation(env.R_W));
        set_actuators(env.name, [0.0; 0.0; 0.0; 0.0])
    end
    
    env.x_W = [0.0; 0.0; 0.0];
    env.v_B = [0.0; 0.0; 0.0];
    env.R_W = Matrix(UnitQuaternion(RotZ(-pi/2.0)*RotY(-pi/2.0)*RotX(pi)));

    env.ω_B = [0.0; 0.0; 0.0];
    env.wind_W = [0.0; 0.0; 0.0];

    env.current_point = 1

    for i in 1:env.n_points
        
        env.desired_length = rand(Uniform(0.1,7))

        if i == 1
            env.desired_angle = rand(Uniform(-pi/2,pi/2))

            env.x_d_W_all[:,i] = env.desired_length*[sin(env.desired_angle); 0.0; cos(env.desired_angle)]
            env.R_d_W_all[:,3*i-2:3*i] = UnitQuaternion(RotY(env.desired_angle)*env.R_W)
        
            if env.visualization
                create_VTOL("fixgoal_$i", actuators = false, color_vec=[0.0; 1.0; 0.0; 1.0]);
                set_transform("fixgoal_$i", env.desired_length*[sin(env.desired_angle); 0.0; cos(env.desired_angle)] ,QuatRotation(env.R_d_W_all[:,3*i-2:3*i])); 
            end
        else
            env.desired_angle = env.desired_angle + rand(Uniform(-pi/2,pi/2))

            env.x_d_W_all[:,i] = env.x_d_W_all[:,i-1] + env.desired_length*[sin(env.desired_angle); 0.0; cos(env.desired_angle)]
            env.R_d_W_all[:,3*i-2:3*i] = UnitQuaternion(RotY(env.desired_angle)*env.R_W)
            
            if env.visualization
                create_VTOL("fixgoal_$i", actuators = false, color_vec=[0.0; 1.0; 0.0; 1.0]);
                set_transform("fixgoal_$i", env.x_d_W_all[:,i-1] + env.desired_length*[sin(env.desired_angle); 0.0; cos(env.desired_angle)] ,QuatRotation(env.R_d_W_all[:,3*i-2:3*i])); 
            end
        end

        
    end
    
    env.x_d_W = [env.x_d_W_all[1,env.current_point]; env.x_d_W_all[2,env.current_point]; env.x_d_W_all[3,env.current_point]]#env.desired_length*[sin(env.desired_angle); 0.0; cos(env.desired_angle)]#env.x_d_W_all[:,1]
    env.R_d_W = env.R_d_W_all[:,3*env.current_point-2:3*env.current_point]
    
    #env.x_d_W = env.desired_length*[sin(env.desired_angle); 0.0; cos(env.desired_angle)]
    #env.R_d_W = UnitQuaternion(RotY(env.desired_angle)*env.R_W)#Matrix(UnitQuaternion(RotZ(calculateAngle([0.0 ,0.0, 1.0], DESIRED_x))*RotZ(-pi/2.0)*RotY(-pi/2.0)*RotX(pi)))

    env.covered_line = 0.0
    env.previously_covered_line = 0.0
    
    env.state = [env.x_W[1];
                 env.x_W[3];
                 env.R_W[1,1];
                 env.R_W[3,1];
                 env.v_B[1];
                 env.v_B[2];    
                 env.ω_B[3];
                 env.x_W[1] - env.x_d_W[1];
                 env.x_W[3] - env.x_d_W[3]; 
                 env.R_d_W[1,1]; 
                 env.R_d_W[3,1];
                 env.covered_line;
                 env.previously_covered_line]
    
    env.t = 0.0
    env.action = [0.0, 0.0]
    env.done = false
    env.cummulative_covered_line = 0.0

    env.reached_goal = false
    nothing

end;

In [ ]:
# defines a methods for a callable object.
# So when a VtolEnv object is created, it has this method that can be called
function (env::VtolEnv)(a)

    # set the propeller trust and the two flaps 2D case
    next_action = [a[1], a[2], 0.0, 0.0]
   
    _step!(env, next_action)
end

In [ ]:
env = VtolEnv();

In [ ]:
methods(env) # Just to explain which methods the object has

In [ ]:
function _step!(env::VtolEnv, next_action)
        
    # caluclate wind impact
    v_in_wind_B = vtol_add_wind(env.v_B, env.R_W, env.wind_W)
    # caluclate aerodynamic forces
    torque_B, force_B = vtol_model(v_in_wind_B, next_action, eth_vtol_param);
    # Limit to 2D
    force_B[3] = 0.0; # Body Z
    env.v_B[3] = 0.0;
    torque_B[1] = 0.0; torque_B[2] = 0.0;  # Body X and Y
    env.ω_B[1] = 0.0; env.ω_B[2] = 0.0;
    # integrate rigid body dynamics for Δt
    env.x_W, env.v_B, env.R_W, env.ω_B, time = rigid_body_simple(torque_B, force_B, env.x_W, env.v_B, env.R_W, env.ω_B, env.t, env.Δt, eth_vtol_param)
    
    # NEW    
    env.covered_line = dot(env.x_W, env.x_d_W)/(norm(env.x_d_W)^2)

    if env.realtime
        sleep(env.Δt) # TODO: just a dirty hack. this is of course slower than real time.
    end

    # Visualize the new state 
    if env.visualization
        set_transform(env.name, env.x_W, QuatRotation(env.R_W));
        set_actuators(env.name, next_action)
    end
 
    env.t += env.Δt
    
    # State space
    #rot = Rotations.params(RotYXZ(env.R_W))[3]
    #env.state[1] = env.x_W[1] # world position in x
    #env.state[2] = env.ω_B[2] # world position in y
    #env.state[3] = rot # rotation around z
    #rot = Rotations.params(RotYXZ(env.R_W))[1]
    
    
    env.state[1] = env.x_W[1];
    env.state[2] = env.x_W[3];
    env.state[3] = env.R_W[1,1];
    env.state[4] = env.R_W[3,1];
    env.state[5] = env.v_B[1];
    env.state[6] = env.v_B[2];
    env.state[7] = env.ω_B[3];
    env.state[8] = env.x_W[1] - env.x_d_W[1];
    env.state[9] = env.x_W[3] - env.x_d_W[3]; 
    env.state[10] = env.R_d_W[1,1]; 
    env.state[11] = env.R_d_W[3,1];
    env.state[12] = env.covered_line;    # Covered distance along line after step
    env.state[13] = env.previously_covered_line; # Covered distance along line before step
    
    
    # Termination criteria
    # TODO: Use many termination criteria so that you do not train unnecessarily in wrong areas

    env.done = #true
        # After time... How fast is drone+Range of desired point
        # After reaching position (circle of r_tol)
        norm(env.ω_B) > 100.0 || 
        norm(env.v_B) > 100.0 || # stop if body is too fast
        env.x_W[3] < -5.0 || # stop if body is below -10m
        #0.0 > rot || # Stop if the drone is pitched 90°.
        #rot > pi || # Stop if the drone is pitched 90°.
        (sum((env.x_W - env.x_d_W).^2) < 0.5 && env.current_point == env.n_points)||
        env.t > env.n_points*10.0# stop after 10s

    if (sum((env.x_W - env.x_d_W).^2) < 0.5 && env.current_point < env.n_points)
        env.reached_goal == false
        env.cummulative_covered_line = env.cummulative_covered_line + norm(env.x_d_W)
        env.current_point = env.current_point + 1
        env.x_d_W = [env.x_d_W_all[1,env.current_point]; env.x_d_W_all[2,env.current_point]; env.x_d_W_all[3,env.current_point]]#env.desired_length*[sin(env.desired_angle); 0.0; cos(env.desired_angle)]#env.x_d_W_all[:,1]
        env.R_d_W = env.R_d_W_all[:,3*env.current_point-2:3*env.current_point]
    end
    nothing
end;

In [ ]:
RLBase.test_runnable!(env)

Show an overview of the environment.

# Setup of a reinforcement learning experiment.

In [ ]:
seed = 123    
rng = StableRNG(seed)
    N_ENV = 8
    UPDATE_FREQ = 1024
    
    
    # define multiple environments for parallel training
    env = MultiThreadEnv([
        # use different names for the visualization
        VtolEnv(; rng = StableRNG(hash(seed+i)), name = "vtol$i") for i in 1:N_ENV
    ])

In [ ]:
# Define the function approximator
    ns, na = length(state(env[1])), length(action_space(env[1]))
    approximator = ActorCritic(
                actor = GaussianNetwork(
                    pre = Chain(
                    Dense(ns, 128, relu; initW = glorot_uniform(rng)),#
                    Dense(128, 128, relu; initW = glorot_uniform(rng)),
                    ),
                    μ = Chain(Dense(128, na; initW = glorot_uniform(rng))),
                    logσ = Chain(Dense(128, na; initW = glorot_uniform(rng))),
                ),
                critic = Chain(
                    Dense(ns, 128, relu; initW = glorot_uniform(rng)),
                    Dense(128, 128, relu; initW = glorot_uniform(rng)),
                    Dense(128, 1; initW = glorot_uniform(rng)),
                ),
                optimizer = ADAM(1e-3),
            );

In [ ]:
    agent = Agent( # A wrapper of an AbstractPolicy
        # AbstractPolicy: the policy to use
        policy = PPOPolicy(;
                    approximator = approximator |> gpu,
                    update_freq=UPDATE_FREQ,
                    dist = Normal,
                    # For parameters visit the docu: https://juliareinforcementlearning.org/docs/rlzoo/#ReinforcementLearningZoo.PPOPolicy
                    ),
        
        # AbstractTrajectory: used to store transitions between an agent and an environment source
        trajectory = PPOTrajectory(;
            capacity = UPDATE_FREQ,
            state = Matrix{Float64} => (ns, N_ENV),
            action = Matrix{Float64} => (na, N_ENV),
            action_log_prob = Vector{Float64} => (N_ENV,),
            reward = Vector{Float64} => (N_ENV,),
            terminal = Vector{Bool} => (N_ENV,),
        ),
    );


In [ ]:
function saveModel(t, agent, env)
    model = cpu(agent.policy.approximator)   
    f = joinpath("./RL_models/", "vtol_2D_ppo_$t.bson")
    @save f model
    println("parameters at step $t saved to $f")
end;

In [ ]:
function loadModel()
    f = joinpath("./RL_models/", "vtol_2D_ppo_2000000.bson") # TODO: evtl anpassen
    @load f model
    return model
end;

In [ ]:
function validate_policy(t, agent, env)
    run(agent.policy, test_env, StopAfterEpisode(1), episode_test_reward_hook)
    # the result of the hook

    # TODO Modify: Not mean, rather last step or last 5 steps
    println("test reward at step $t: $(episode_test_reward_hook.rewards[end])")
    
end;

episode_test_reward_hook = TotalRewardPerEpisode(;is_display_on_exit=false)
# create a env only for reward test
test_env = VtolEnv(;name = "testVTOL", visualization = true, realtime = true);

In [ ]:
#agent.policy.approximator = loadModel();

In [ ]:
ReinforcementLearning.run(
    agent,
    env,
    StopAfterStep(1_500_000),
    ComposedHook(
        DoEveryNStep(saveModel, n=100_000), 
        DoEveryNStep(validate_policy, n=10_000)),
)

In [ ]:
plot(episode_test_reward_hook.rewards)

In [ ]:
close_visualization(); # closes the MeshCat visualization